In [23]:
import requests
import pandas as pd
import os
import json
import mysql.connector

# COLLECTING DATA

In [24]:
api_key = os.getenv('COINCAP')

In [25]:
url = "https://api.coincap.io/v2/assets"

headers = {
    "Authorization": api_key,
    "Content-Type": "application/json"
}

response = requests.get(url=url, headers=headers)

In [26]:
response_json = json.loads(response.text)

In [27]:
df = pd.DataFrame(response_json['data'])
df.head()

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,19629387.0000000000000000,21000000.0000000000000000,1028258023842.9507161182487283,18259967454.5224856663386622,52383.6034127275964409,0.1842850283714662,52056.4579358706336320,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,120165596.0899642900000000,None,341804352385.8146699444355241,10057780656.3668913484317424,2844.4443626769533304,1.3016925516340193,2831.2295438622524821,https://etherscan.io/
2,tether,3,USDT,Tether,97116788125.7176400000000000,None,97197350540.1491441982170850,19179526928.1145048382958220,1.0008295415858195,-0.0120074888223146,1.0007774566556557,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,BNB,166801148.0000000000000000,166801148.0000000000000000,60446848583.6215701169103064,681047622.0656101265589498,362.3886844209343818,4.1005594787382146,357.6990593014604770,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,5,SOL,Solana,440507299.8164372400000000,None,49698565136.7815038669264045,717633454.4753330084926142,112.8212067257257166,-2.6748721959538936,113.7891062863362846,https://explorer.solana.com/


In [28]:
df = df.drop(columns=['rank', 'supply', 'maxSupply', 'marketCapUsd', 'volumeUsd24Hr', 'priceUsd', 'changePercent24Hr', 'vwap24Hr'])
df.head()

,id,symbol,name,explorer
0,bitcoin,BTC,Bitcoin,https://blockchain.info/
1,ethereum,ETH,Ethereum,https://etherscan.io/
2,tether,USDT,Tether,https://www.omniexplorer.info/asset/31
3,binance-coin,BNB,BNB,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,SOL,Solana,https://explorer.solana.com/


In [29]:
new_column_names = {'id':'asset_id', 'symbol':'asset_symbol', 'name':'asset_name', 'explorer':'asset_info_link'}
df = df.rename(columns=new_column_names)

In [30]:
df.head()

,asset_id,asset_symbol,asset_name,asset_info_link
0,bitcoin,BTC,Bitcoin,https://blockchain.info/
1,ethereum,ETH,Ethereum,https://etherscan.io/
2,tether,USDT,Tether,https://www.omniexplorer.info/asset/31
3,binance-coin,BNB,BNB,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,SOL,Solana,https://explorer.solana.com/


# LOADING DATA INTO MYSQL

In [31]:
mysql_password = os.getenv('MYSQL_PASSWORD')

mysql_connection = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = mysql_password,
  database = 'cripto_infos'
)

In [32]:
cursor = mysql_connection.cursor()

In [33]:
for index, row in df.iterrows():
    data = (row['asset_id'], row['asset_symbol'], row['asset_name'], row['asset_info_link'])
    query = "INSERT assets (asset_id, asset_symbol, asset_name, asset_info_link) VALUES (%s, %s, %s, %s)"
    cursor.execute(query, data)

In [34]:
mysql_connection.commit()
mysql_connection.close()